# Load data in pandas

In [0]:
test_data_a = "https://drive.google.com/uc?id=1rbhlUfffty5pHWPMPKBhR76dz4jp7ZFb&export=download"

train_data_a = "https://drive.google.com/uc?id=1UU5wVku8v79aEKknkOBMrKNDZrTxUpJZ&export=download"

test_data = "https://drive.google.com/uc?id=1Yabjq972kr7pA-HST7MP-9gR0xTSOjs_&export=download"

train_data = "https://drive.google.com/uc?id=1SButl51QPhJ0WbFniXFLFO-YoxzJ0V3p&export=download"

valid_data = "https://drive.google.com/uc?id=17EONLP166qnOeedOZrIl5xu5zznRH9b3&export=download"

valid_data_a = "https://drive.google.com/uc?id=11Txw4zCcyYCJ6TCsr90F-LF2_b63HiZ2&export=download"

In [2]:
!pip install pandas==0.24.1 # Google colab pandas is too old to read these pickle
!pip install scipy==1.2.1 # Scipy is too low to have the function

In [0]:
def download(url):
  import requests
  from io import BytesIO
  r = requests.get(url)
  data = r.content
  df = pd.read_pickle(BytesIO(data),compression = None)
  return df

In [0]:
import pandas as pd


test_data = download(test_data)
test_data_a = download(test_data_a)
train_data = download(train_data)
train_data_a = download(train_data_a)
valid_data = download(valid_data)
valid_data_a = download(valid_data_a)

In [5]:
train_data.columns

Index(['ID', 'Text', 'Pronoun', 'Pronoun-offset', 'A', 'A-offset', 'A-coref',
       'B', 'B-offset', 'B-coref', 'URL', 'bert_tokens_torch', 'name_list',
       'token_map_bert', 'sentence_map_bert', 'A_dist_bert', 'B_dist_bert',
       'A_pos_bert', 'B_pos_bert', 'pron_pos_bert', 'A_idx_bert_torch',
       'B_idx_bert_torch', 'pron_idx_bert_torch', 'topic', 'topic_A',
       'topic_B', 'name_idx_bert_torch', 'neither_list',
       'neither_idx_bert_torch', 'label'],
      dtype='object')

# Fine-tune BERT-large and mlp model

In [6]:
!pip install pytorch-pretrained-bert
import numpy as np
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel
import torch.nn as nn

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
# form train_data
train_all = pd.concat([train_data, valid_data], ignore_index=True, sort=False)
# train_all = train_data
train_all['indexed_tokens'] = train_all['bert_tokens_torch'].apply(lambda x: tokenizer.convert_tokens_to_ids(x))
train_indexed_tokens = list(train_all['indexed_tokens'])
train_offsets_A = list(train_all['A_idx_bert_torch'])
train_offsets_B = list(train_all['B_idx_bert_torch'])
train_offsets_pron = list(train_all['pron_idx_bert_torch'])
train_offsets_other = list(train_all['neither_idx_bert_torch'])
train_offsets_cls = [np.array([0])]*len(train_offsets_other)
train_labels = list(train_all['label'])
print('Number of training data is:',len(train_all))


# form test_data
test_data['indexed_tokens'] = test_data['bert_tokens_torch'].apply(lambda x: tokenizer.convert_tokens_to_ids(x))
test_indexed_tokens = list(test_data['indexed_tokens'])
test_offsets_A = list(test_data['A_idx_bert_torch'])
test_offsets_B = list(test_data['B_idx_bert_torch'])
test_offsets_pron = list(test_data['pron_idx_bert_torch'])
test_offsets_other = list(test_data['neither_idx_bert_torch'])
test_offsets_cls = [np.array([0])]*len(test_offsets_other)
test_labels = list(test_data['label'])
print('Number of testing data is:',len(test_data))

Number of training data is: 2454
Number of testing data is: 2000


In [0]:
import numpy as np

torch_rd = 78
np.random.seed(seed=0)

import torch
torch.manual_seed(torch_rd)

model_name = "bert_large_rd"+str(torch_rd)

CV = 6

In [9]:
max([len(i) for i in test_indexed_tokens])

348

In [0]:
from sklearn.model_selection import KFold 
kf = KFold(n_splits=5) 
k = 0
for train_index, test_index in kf.split(train_labels):
  if k == CV:
    break
  k = k + 1

In [11]:
test_index

array([1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974,
       1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985,
       1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996,
       1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007,
       2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018,
       2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029,
       2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040,
       2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051,
       2052, 2053, 2054, 2055, 2056, 2057, 2058, 2059, 2060, 2061, 2062,
       2063, 2064, 2065, 2066, 2067, 2068, 2069, 2070, 2071, 2072, 2073,
       2074, 2075, 2076, 2077, 2078, 2079, 2080, 2081, 2082, 2083, 2084,
       2085, 2086, 2087, 2088, 2089, 2090, 2091, 2092, 2093, 2094, 2095,
       2096, 2097, 2098, 2099, 2100, 2101, 2102, 2103, 2104, 2105, 2106,
       2107, 2108, 2109, 2110, 2111, 2112, 2113, 21

In [0]:
if CV in [0,1,2,3,4]:
  train_indexed_tokens,test_indexed_tokens = [train_indexed_tokens[i] for i in list(train_index)],[train_indexed_tokens[i] for i in list(test_index)]
  train_offsets_A,test_offsets_A = [train_offsets_A[i] for i in list(train_index)],[train_offsets_A[i] for i in list(test_index)]
  train_offsets_B,test_offsets_B = [train_offsets_B[i] for i in list(train_index)],[train_offsets_B[i] for i in list(test_index)]
  train_offsets_pron,test_offsets_pron = [train_offsets_pron[i] for i in list(train_index)],[train_offsets_pron[i] for i in list(test_index)]
  train_offsets_other,test_offsets_other = [train_offsets_other[i] for i in list(train_index)],[train_offsets_other[i] for i in list(test_index)]
  train_offsets_cls,test_offsets_cls = [train_offsets_cls[i] for i in list(train_index)],[train_offsets_cls[i] for i in list(test_index)]
  train_labels,test_labels = [train_labels[i] for i in list(train_index)],[train_labels[i] for i in list(test_index)]
  
if CV == 6:
  train_indexed_tokens = train_indexed_tokens + test_indexed_tokens
  train_offsets_A = train_offsets_A + test_offsets_A
  train_offsets_B = train_offsets_B + test_offsets_B
  train_offsets_pron = train_offsets_pron + test_offsets_pron
  train_offsets_other = train_offsets_other + test_offsets_other
  train_offsets_cls = train_offsets_cls + test_offsets_cls
  train_labels = train_labels + test_labels
  

In [0]:
class MLP(nn.Module):
    def __init__(self, bert_hidden_size):
        super().__init__()
        self.bert_hidden_size = bert_hidden_size
        self.fc = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(bert_hidden_size, 1),

        )
        
    def forward(self, vector):
        return self.fc(vector)

In [0]:
def change_dropout(module,p):
    for name in module.__dict__['_modules']:
        if name == "dropout":
            module.__dict__['_modules'][name].p = p
        else:
            change_dropout(module.__dict__['_modules'][name],p)

In [0]:
def extract_vectors(bert_last_layer, offsets, offsets_mask):
    extracted_vectors = bert_last_layer.gather(
        1, offsets.unsqueeze(2).expand(-1, -1, 1))
    vector_masked = extracted_vectors.mul(offsets_mask.unsqueeze(2).float()).sum(1)/(offsets_mask.sum(1).unsqueeze(1).float())
    return vector_masked
  
def get_padded_and_mask(sequence, left, right):
    seq_ori = sequence[left:right]
    seq_padded = pad_sequence([torch.Tensor(v) for v in seq_ori],batch_first = True).cuda().long()
    mask = (seq_padded != 0).cuda().long()
    return seq_padded, mask

In [16]:
from torch.nn.utils.rnn import pad_sequence
sub_batch = 2 # batch size越大越差？不知道什么操作
acc_time = 8
acc_counter = 0
EPOCHS = 1
  

loss_c = nn.CrossEntropyLoss()
loss_c.cuda()
bert = BertModel.from_pretrained('bert-large-uncased')
bert =  bert.cuda()
change_dropout(bert,0.15) # 上面28的情况下改成dropout 0.1会爆内存

mlp = MLP(1024).cuda()
change_dropout(mlp,0)

#opt = torch.optim.Adam([
#     {'params': bert.parameters(), 'lr': 2e-5},{'params': mlp.parameters()}
# ], lr=1e-3)
# opt = torch.optim.SGD(list(bert.parameters()) + list(mlp.parameters()), lr = 2e-5, momentum = 2e-4)
opt = torch.optim.Adam(list(bert.parameters()) + list(mlp.parameters()), lr=2e-5)

for e in range(1, EPOCHS+1):
  for b in range(0,len(train_offsets_cls),sub_batch):
    start = b
    end = min(len(train_offsets_cls),b+sub_batch)
  
    tr_X_padded, attention_mask = get_padded_and_mask(train_indexed_tokens, start, end)
    tr_Y = torch.tensor(train_labels[start:end]).cuda()
    tr_A_padded, tr_A_mask = get_padded_and_mask(train_offsets_A, start, end)
    tr_B_padded, tr_B_mask = get_padded_and_mask(train_offsets_B, start, end)
    tr_other_padded, tr_other_mask = get_padded_and_mask([np.array(list(i)+list(j)) for i,j in zip(train_offsets_cls,train_offsets_pron)], start, end)
    segments_tensors = torch.zeros(tr_X_padded.size()).cuda().long()
    
    encoded_bert, _ = bert(tr_X_padded, segments_tensors,attention_mask,output_all_encoded_layers=False)
    
    pred_ = mlp(encoded_bert)
    pred = torch.cat([extract_vectors(pred_, tr_A_padded, tr_A_mask),\
                              extract_vectors(pred_, tr_B_padded, tr_B_mask),extract_vectors(pred_, tr_other_padded, tr_other_mask)],1)

    loss = loss_c(pred, tr_Y)

    loss.backward()
    acc_counter = acc_counter+1
    if acc_counter == acc_time:
      opt.step()
      acc_counter = 0
      opt.zero_grad()
      torch.cuda.empty_cache()
    if b%100 == 0:
      print ("Epoch =",e,"loss = ",loss.item())

Epoch = 1 loss =  1.1234710216522217
Epoch = 1 loss =  0.589855968952179
Epoch = 1 loss =  0.8457642793655396
Epoch = 1 loss =  0.6831784248352051
Epoch = 1 loss =  0.29253000020980835
Epoch = 1 loss =  0.24752283096313477
Epoch = 1 loss =  0.583827793598175
Epoch = 1 loss =  0.2646169662475586
Epoch = 1 loss =  0.15187382698059082
Epoch = 1 loss =  1.9655089378356934
Epoch = 1 loss =  0.3362148404121399
Epoch = 1 loss =  0.5149853229522705
Epoch = 1 loss =  1.91571044921875
Epoch = 1 loss =  0.12521660327911377
Epoch = 1 loss =  0.26120948791503906
Epoch = 1 loss =  0.558201789855957
Epoch = 1 loss =  0.7934609055519104
Epoch = 1 loss =  0.056888699531555176
Epoch = 1 loss =  0.48524463176727295
Epoch = 1 loss =  2.0962138175964355
Epoch = 1 loss =  0.7157219052314758
Epoch = 1 loss =  0.7790604829788208
Epoch = 1 loss =  0.9454501271247864
Epoch = 1 loss =  0.08101242780685425
Epoch = 1 loss =  0.8946458697319031
Epoch = 1 loss =  0.70745849609375
Epoch = 1 loss =  0.4614053964614868

In [17]:
re = []
bert.eval()
mlp.eval()
with torch.no_grad():
  for b in range(0,len(test_offsets_cls),1):
    start = b
    end = min(len(test_offsets_cls),b+1)
    tr_X_padded, attention_mask = get_padded_and_mask(test_indexed_tokens, start, end)
    tr_Y = torch.tensor(test_labels[start:end]).cuda()
    tr_A_padded, tr_A_mask = get_padded_and_mask(test_offsets_A, start, end)
    tr_B_padded, tr_B_mask = get_padded_and_mask(test_offsets_B, start, end)
    tr_other_padded, tr_other_mask = get_padded_and_mask([np.array(list(i)+list(j)) for i,j in zip(test_offsets_cls,test_offsets_pron)], start, end)
    segments_tensors = torch.zeros(tr_X_padded.size()).cuda().long()
    
    encoded_bert, _ = bert(tr_X_padded, segments_tensors,attention_mask,output_all_encoded_layers=False)
    
    pred_ = mlp(encoded_bert)
    pred = torch.cat([extract_vectors(pred_, tr_A_padded, tr_A_mask),\
                              extract_vectors(pred_, tr_B_padded, tr_B_mask),extract_vectors(pred_, tr_other_padded, tr_other_mask)],1)
    pred_data = pred.data.cpu().numpy()
    re.append(pred_data)

loss_c = nn.CrossEntropyLoss()

loss_data = loss_c(torch.Tensor(np.concatenate(re).reshape(-1,3)),torch.Tensor(test_labels).long())
print('Loss for test data is:', loss_data)

Loss for test data is: tensor(0.1678)


In [0]:
import os
from joblib import dump, load
from scipy.special import softmax
pred = np.zeros((len(train_all),3))

if CV in [0,1,2,3,4]:
  if os.path.exists(model_name+'+basic.joblib'):
    pred = load(model_name+'+basic.joblib')
    os.remove(model_name+'+basic.joblib')
  pred[test_index,:] = softmax(np.concatenate(re).reshape(-1,3),axis = 1)
  dump(pred,model_name+'+basic.joblib')

if CV == 5:
  if os.path.exists(model_name+'+basic.joblib'):
    pred_tr = load(model_name+'+basic.joblib')
    os.remove(model_name+'+basic.joblib')
  pred = softmax(np.concatenate(re).reshape(-1,3),axis = 1)
  dump((pred_tr,pred),model_name+'+basic.joblib')
  
if CV == 6:
  dump((bert,mlp),model_name +'.joblib')
  


  
  

      
      